<a href="https://colab.research.google.com/github/Elwing-Chou/ml0930/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0930/raw/master/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0930/raw/master/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
datas = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [ ]:
datas = datas.drop(["PassengerId", "Survived"], axis=1)

In [ ]:
na = datas.isna().sum()
# 篩選 Series[True/False list/series]
na[na > 0].sort_values(ascending=False)

In [ ]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 先處理一下ticket
dic = datas["Ticket"].value_counts()
def count(n):
    return dic[n]
datas["Ticket"] = datas["Ticket"].apply(count)

In [ ]:
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)

In [ ]:
# 一個段落, 看一下還缺啥
na = datas.isna().sum()
# 篩選 Series[True/False list/series]
na[na > 0].sort_values(ascending=False)